In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np
%matplotlib inline
plt.rcParams['font.size'] = 16

In [2]:
# read csv files
file_dir = '../csv_files/'
train_set = 'pinder'
test_set = 'db5_ab_ag'
model = 'model_1'
num_samples = 40
num_steps = 40

df_list = []
for i in range(1,3):
    file_name = f'{test_set}_{model}_{num_samples}_samples_{num_steps}_steps_{train_set}_{i}'
    df = pd.read_csv(f'{file_dir}{file_name}.csv')
    df['run'] = i
    df_list.append(df)

In [3]:
df_sr_list = []
for df in df_list:
    result_df = df.copy().sort_values(by='energy', ascending=True).groupby('id')
    top1 = result_df.head(1).copy()
    top5 = result_df.head(5).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
    top10 = result_df.head(10).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
    top25 = result_df.head(25).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
    top40 = result_df.head(40).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
    best = df.sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
    # merge dataframes
    top1['method'] = 'Top-1'
    top5['method'] = 'Top-5'
    top10['method'] = 'Top-10'
    top25['method'] = 'Top-25'
    best['method'] = 'Oracle'

    # merge the dataframes 
    df_merged = pd.concat([top1, top5, top10, top25, best])

    # get the order
    cols = ['Top-1', 'Top-5', 'Top-10', 'Top-25', 'Oracle']

    df_merged['method'] = pd.Categorical(df_merged['method'], categories=cols, ordered=True)

    # separate for each metric
    df_DockQ = df_merged.pivot(index='id', columns='method', values='DockQ')[cols]

    df_sr = pd.DataFrame()
    df_sr['Acceptable'] = df_DockQ[(df_DockQ >= 0.23)].count() / df_DockQ.shape[0]
    df_sr['Medium'] = df_DockQ[(df_DockQ >= 0.49)].count() / df_DockQ.shape[0]
    df_sr['High'] = df_DockQ[(df_DockQ >= 0.8)].count() / df_DockQ.shape[0]
    df_sr = df_sr.reset_index()
    df_sr_list.append(df_sr)

df_sr = pd.concat(df_sr_list)

In [4]:
df = pd.concat(df_list)
result_df = df.copy().sort_values(by='energy', ascending=True).groupby('id')
top1 = result_df.head(1).copy()
top5 = result_df.head(5).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
top10 = result_df.head(10).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
top25 = result_df.head(25).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
top40 = result_df.head(40).sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
best = df.sort_values(by='DockQ', ascending=False).groupby('id').head(1).copy()
# merge dataframes
top1['method'] = 'Top-1'
top5['method'] = 'Top-5'
top10['method'] = 'Top-10'
top25['method'] = 'Top-25'
best['method'] = 'Oracle'

# merge the dataframes 
df_merged = pd.concat([top1, top5, top10, top25, best])

# get the order
cols = ['Top-1', 'Top-5', 'Top-10', 'Top-25', 'Oracle']

df_merged['method'] = pd.Categorical(df_merged['method'], categories=cols, ordered=True)

# separate for each metric
df_DockQ = df_merged.pivot(index='id', columns='method', values='DockQ')[cols]

df_sr = pd.DataFrame()
df_sr['Acceptable'] = df_DockQ[(df_DockQ >= 0.23)].count() / df_DockQ.shape[0]
df_sr['Medium'] = df_DockQ[(df_DockQ >= 0.49)].count() / df_DockQ.shape[0]
df_sr['High'] = df_DockQ[(df_DockQ >= 0.8)].count() / df_DockQ.shape[0]
df_sr = df_sr.reset_index()

In [5]:
melted_df = pd.melt(df_sr, id_vars='method', var_name='Quality', value_name='Success Rate')

# Define custom colors for pastel red, muted red, and dark red
custom_palette = ["#FFC3A0", "#FF5733", "#8B0000"]

g = sns.catplot(
    data=melted_df, 
    kind='bar', 
    x='method', 
    y='Success Rate', 
    hue='Quality', 
    dodge=False, 
    palette=custom_palette,
    height=6,
    aspect=1,
    legend_out=False,
) 

g.set_xlabels("")
g.ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

#plt.ylim(0, 0.5)
plt.title("Ab-Ag set from AlphaRED \n (N=67)")
plt.xticks(rotation=0)
plt.savefig('DFMDock_db5_ab_ag.png', dpi=300, bbox_inches="tight")
plt.show()

In [6]:
best

id  index     c_rmsd     i_rmsd     l_rmsd      fnat     DockQ  \
2594  6DBG     34   1.523864   1.188302   5.508422  1.000000  0.772883   
494   4POU     14   2.684352   1.727191   6.927492  1.000000  0.676943   
1665  3U7Y     25   2.540024   1.531252   7.552685  0.928571  0.659023   
1889  4G6M      9   1.597480   1.928465   3.546731  0.666666  0.631775   
1612  3SE8     12   2.842550   1.701803   7.922655  0.666667  0.546334   
...    ...    ...        ...        ...        ...       ...       ...   
1509  3MJ9     29  15.103190  13.090083  30.086712  0.000000  0.028959   
1080  1S78      0   9.228056  12.468606  31.857487  0.000000  0.026908   
1469  3HMX     29  19.356483  16.272429  34.904461  0.000000  0.021470   
737   5VNW     17  12.199641   8.362682  75.049629  0.000000  0.014612   
1926  4GXU      6  33.995552  24.400631  55.538582  0.000000  0.008884   

         energy  num_clashes  run  method  
2594 -21.632822            0    2  Oracle  
494  -14.977560            3    2  Oracle  
1665 -18.199120            0    2  Oracle  
1889 -12.409532            1    1  Oracle  
1612 -19.180635            1    2  Oracle  
...         ...          ...  ...     ...  
1509 -12.515967            2    1  Oracle  
1080  -8.021659            3    1  Oracle  
1469  -7.714610            7    2  Oracle  
737   -6.092248           43    1  Oracle  
1926  -7.147402            1    2  Oracle  

[66 rows x 11 columns]

In [7]:
top1[top1['id'] == '1WEJ']

id  index    c_rmsd     i_rmsd     l_rmsd  fnat     DockQ     energy  \
1163  1WEJ      3  9.555011  10.706242  22.564039   0.0  0.047841 -17.309799   

      num_clashes  run method  
1163            0    1  Top-1

In [8]:
filtered_best = best[best['DockQ'] > 0.23]

sns.scatterplot(data=filtered_best, x='id', y='DockQ')
plt.axhline(y=0.23, color='grey', linestyle='--', alpha=0.5, lw=2.0)
plt.axhline(y=0.49, color='grey', linestyle='--', alpha=0.5, lw=2.0)
plt.xticks(rotation=90)  # Rotate x-ticks by 90 degrees
plt.ylim(0, 1)
plt.show()